## Data Mining project: Discover and describe areas of interest an events from geo-located data

## Task #1: Import Dataset and Librarie

In [71]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3 

In [72]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:


# Charger le fichier CSV
data = pd.read_csv("../data/dataSet.csv", sep=",", low_memory=False)

# Sélectionner les colonnes 11 et 12 (indices 10 et 11 en pandas)
columns_to_check = data.iloc[:, [11, 12]]

# Parcourir chaque colonne sélectionnée
for col in columns_to_check.columns:
    print(f"Analyse de la colonne: {col}")
    
    # Trouver les types de données présents
    types_present = data[col].map(type).value_counts()
    print("Types présents dans cette colonne :")
    print(types_present)
    
    # Afficher l'utilisation de la mémoire pour cette colonne
    memory_usage = data[col].memory_usage(deep=True)
    print(f"Utilisation de la mémoire pour cette colonne : {memory_usage} bytes\n")

for col in columns_to_check.columns:
    print(f"Conversion de la colonne: {col}")
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)  # Convertir en float, remplacer les erreurs par NaN

    # Vérifier le résultat
    print(f"Colonne {col} après conversion:")
    print(data[col].head())  # Afficher les premières lignes pour valider
    print(f"Types après conversion: {data[col].map(type).value_counts()}\n")

Analyse de la colonne:  date_upload_minute
Types présents dans cette colonne :
 date_upload_minute
<class 'str'>      420228
<class 'float'>        12
Name: count, dtype: int64
Utilisation de la mémoire pour cette colonne : 21366572 bytes

Analyse de la colonne:  date_upload_hour
Types présents dans cette colonne :
 date_upload_hour
<class 'str'>      420238
<class 'float'>         2
Name: count, dtype: int64
Utilisation de la mémoire pour cette colonne : 21349400 bytes

Conversion de la colonne:  date_upload_minute
Colonne  date_upload_minute après conversion:
0    23.0
1    52.0
2    33.0
3    38.0
4    38.0
Name:  date_upload_minute, dtype: float64
Types après conversion:  date_upload_minute
<class 'float'>    420240
Name: count, dtype: int64

Conversion de la colonne:  date_upload_hour
Colonne  date_upload_hour après conversion:
0    20.0
1    17.0
2    17.0
3    12.0
4    12.0
Name:  date_upload_hour, dtype: float64
Types après conversion:  date_upload_hour
<class 'float'>    4202

In [74]:
#rename columms
data.columns = data.columns.str.strip()
print("Colonnes après nettoyage :", data.columns.tolist())


Colonnes après nettoyage : ['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute', 'date_taken_hour', 'date_taken_day', 'date_taken_month', 'date_taken_year', 'date_upload_minute', 'date_upload_hour', 'date_upload_day', 'date_upload_month', 'date_upload_year', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']


In [ ]:
#Find null values
print(f"Initial: {len(data)}")
data.isna().sum()

Initial: 420240


id                         0
user                       0
lat                        0
long                       0
tags                  103510
title                  38329
date_taken_minute          1
date_taken_hour            0
date_taken_day             0
date_taken_month           0
date_taken_year            0
date_upload_minute         0
date_upload_hour           0
date_upload_day            2
date_upload_month          0
date_upload_year           1
Unnamed: 16           420098
Unnamed: 17           420240
Unnamed: 18           420238
dtype: int64

In [68]:
print(f"Initial: {len(data)}")
print(data.columns)
data_cleaned = data.dropna(subset=['lat', 'long'])
print(f"After removing missing values: {len(data_cleaned)}")

Initial: 420240
Index(['id', 'user', 'lat', 'long', 'tags', 'title', 'date_taken_minute',
       'date_taken_hour', 'date_taken_day', 'date_taken_month',
       'date_taken_year', 'date_upload_minute', 'date_upload_hour',
       'date_upload_day', 'date_upload_month', 'date_upload_year',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')
After removing missing values: 420240


# Duplicates

In [70]:
data_cleaned = data_cleaned.drop_duplicates(subset=['user','lat','long','title','date_taken_year','date_taken_month','date_taken_day'],keep='first')
print(f"Initial: {len(data)}")
print(f"After removing duplicates: {len(data_cleaned)}")

Initial: 420240
After removing duplicates: 133925
